# Notebook for identifying the most relevant features in my dataset

In [104]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer
from scipy.stats import anderson
import pandas as pd
import geopandas as gpd
import numpy as np

In [105]:
data = gpd.read_file('../data/shapes/censCrashes.shp')
data.columns.tolist()
irrelevant = ['YEAR', 'STATE', 'COUNTY', 'geometry', 'STATEA', 'COUNTYA', 'TRACTA', 'AREANAM', ]
target='grade'

data.set_index("GISJOIN", inplace=True)

In [106]:
pred_data = data.copy().drop(columns=irrelevant)

pred_data.head()

,popTotl,popWhit,ppNnwht,ngrPpTt,occDwUT,mlNSchl,mlElm14,mlElm56,mlElm78,mlHS1t3,...,rfrgMch,refrgIc,rfrgOth,refrgNn,rfrgNRp,htCntrl,htNCntr,htNRprt,grade,n_crashes
GISJOIN,,,,,,,,,,,,,,,,,,,,,
G06003700001,4304.0,4069.0,235.0,70.0,874.0,62.0,114.0,124.0,380.0,271.0,...,571.0,165.0,2.0,132.0,4.0,82.0,784.0,2.0,C,3053.0
G06003700004,7248.0,6919.0,329.0,34.0,1927.0,73.0,314.0,204.0,564.0,360.0,...,842.0,654.0,27.0,389.0,15.0,62.0,1844.0,5.0,C,4293.0
G06003700006,864.0,789.0,75.0,2.0,265.0,4.0,20.0,7.0,73.0,53.0,...,195.0,36.0,6.0,25.0,3.0,22.0,239.0,1.0,C,250.0
G06003700007,2925.0,2886.0,39.0,0.0,893.0,17.0,38.0,58.0,294.0,157.0,...,503.0,291.0,7.0,85.0,7.0,18.0,867.0,2.0,C,502.0
G06003700012,607.0,600.0,7.0,1.0,208.0,1.0,15.0,14.0,53.0,43.0,...,111.0,68.0,1.0,27.0,1.0,1.0,207.0,0.0,C,96.0


In [107]:
expo, norm = 0, 0
for column in pred_data.drop(columns=['grade', 'n_crashes']).columns:
    result = anderson(pred_data[column], dist='expon')
    if result.statistic > result.critical_values[2]:
        # print(f'Feature {column} had {result.statistic} > {result.critical_values[2]} anderson')
        expo += 1
    result = anderson(pred_data[column], dist='expon')
    if result.statistic > result.critical_values[2]:
        norm += 1

print(f'Around {expo / (len(pred_data.columns)-1) * 100} of data is non-exponential; {expo} features')
print(f'Around {norm / (len(pred_data.columns)-1) * 100} of data is non-exponential; {norm} features')

Around 99.04761904761905 of data is non-exponential; 104 features
Around 99.04761904761905 of data is non-exponential; 104 features


Oh wonderful. Everything is non-normal _and_ non-exponential. Let's just use min-max scaler for the preprocessing, then.

In [108]:
X = pred_data.copy().drop(columns=['grade', 'n_crashes'])
X_scaled = MinMaxScaler().fit_transform(X)

X_scaled

array([[0.15712617, 0.1491077 , 0.01490266, ..., 0.01155743, 0.18148148,
        0.02173913],
       [0.2646028 , 0.25354538, 0.02086372, ..., 0.00873855, 0.42685185,
        0.05434783],
       [0.03154206, 0.02891275, 0.00475617, ..., 0.00310078, 0.05532407,
        0.01086957],
       ...,
       [0.10517669, 0.10557367, 0.        , ..., 0.07667371, 0.0150463 ,
        0.        ],
       [0.18100175, 0.18139177, 0.00050732, ..., 0.16659619, 0.01203704,
        0.        ],
       [0.0239486 , 0.02403899, 0.        , ..., 0.01508104, 0.00787037,
        0.        ]])

In [109]:
y = pred_data.grade.apply(lambda x: 'AB' if x in ['A', 'B'] else 'CD')

In [110]:
logreg = LogisticRegression('l1', solver='saga', max_iter=10000)
logreg.fit(X, y).score(X, y)

0.875438596491228

In [111]:
logreg.feature_names_in_

array(['popTotl', 'popWhit', 'ppNnwht', 'ngrPpTt', 'occDwUT', 'mlNSchl',
       'mlElm14', 'mlElm56', 'mlElm78', 'mlHS1t3', 'maleHS4', 'mlCll13',
       'mlCllg4', 'mlNSchR', 'fmlNSch', 'fmlEl14', 'fmlEl56', 'fmlEl78',
       'fmlHS13', 'femlHS4', 'fmlCl13', 'fmlCll4', 'fmlNScR', 'mlMdnYr',
       'fmlMdnY', 'mlInLbr', 'mlNtInL', 'fmlInLb', 'fmlNtIL', 'mlEmply',
       'mlPbEmW', 'mlSkWrk', 'mlNtILH', 'mlNtILS', 'mlNtILU', 'mlNtILI',
       'mlNtILO', 'fmlEmpl', 'fmlPbEW', 'fmlSkWr', 'fmlNILH', 'fmlNILS',
       'fmlNILU', 'fmlNILI', 'fmlNILO', 'malePrf', 'mlSmPrf', 'malePrp',
       'mlClrcl', 'mlCrfts', 'mlOprtv', 'mlDmstc', 'malSrvc', 'maleLbr',
       'mlNOccR', 'femlPrf', 'fmlSmPr', 'femlPrp', 'fmlClrc', 'fmlCrft',
       'fmlOprt', 'fmlDmst', 'fmlSrvc', 'femlLbr', 'fmlNOcR', 'fm1Dtch',
       'fm1Attc', 'fm2SdBS', 'fm2Othr', 'fam3', 'fam4', 'fm1t4WB',
       'fam5to9', 'fm10t19', 'fm20pls', 'othrStr', 'undrP51', 'pt51t75',
       'pt76to1', 'pt1t1p5', 'pt1p5t2', 'pt2plus', 'nRprt

In [112]:
logreg.coef_.shape

(1, 104)

In [113]:
coefs = pd.DataFrame(logreg.coef_.T, index=logreg.feature_names_in_, columns=['coef'])\
    .reset_index()\
    .rename(columns={'index': 'feature'})


high, low = coefs.sort_values(by='coef').head(5), coefs.sort_values(by='coef', ascending=False).head(5)
high, low

(    feature      coef
 18  fmlHS13 -0.004170
 47  malePrp -0.003501
 67  fm2SdBS -0.003390
 52  malSrvc -0.003371
 61  fmlDmst -0.002861,
     feature      coef
 70     fam4  0.004902
 60  fmlOprt  0.003409
 97  refrgIc  0.002923
 50  mlOprtv  0.002887
 26  mlNtInL  0.002619)

In [116]:
coefs.coef.describe()

count    104.000000
mean       0.000180
std        0.001460
min       -0.004170
25%       -0.000595
50%        0.000236
75%        0.000991
max        0.004902
Name: coef, dtype: float64

Ooookay, this is a little hard to interpret. Let's try something more human: how much variance can a model with only these 20 features explain?

In [ ]:
rel = high.feature.tolist() + low.feature.tolist()

X = pred_data[rel]
X_scaled = MinMaxScaler().fit_transform(X)

logreg.fit(X_scaled, y)
logreg.score(X_scaled, y)

0.8258771929824561

In [118]:
logreg.coef_, rel

(array([[-2.34722755, -6.40581338, -4.71938197,  2.42852676, -3.70271066,
         12.21897839,  1.48022964, 15.84593785,  1.41913549,  0.        ]]),
 ['fmlHS13',
  'malePrp',
  'fm2SdBS',
  'malSrvc',
  'fmlDmst',
  'fam4',
  'fmlOprt',
  'refrgIc',
  'mlOprtv',
  'mlNtInL'])